In [1]:
# !pip install -r requirements.txt

In [2]:
import calitp
from calitp.tables import tbl
from siuba import *

import pandas as pd
import numpy as np
import geopandas as gpd
import fiona
import shapely

from ipyleaflet import Map, GeoJSON, projections, basemaps, GeoData, LayersControl, WidgetControl, GeoJSON
from ipywidgets import Text, HTML

from utilities import *

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [161]:
bus_hqtc = gpd.read_parquet(f'{GCS_FILE_PATH}intermediate/shape_dissolve.parquet')
bus_hqtc = bus_hqtc[bus_hqtc['hq_transit_corr']]
bus_hqtc['hqta_type'] = 'hq_transit_corr'
bus_hqtc['route_type'] = '3'

In [4]:
rail_ferry_brt_stops = gpd.read_parquet(f'{GCS_FILE_PATH}rail_brt_ferry.parquet')
rail_ferry_brt_stops['hqta_type'] = 'major_transit_stop'

In [5]:
rail_ferry_brt_stops.columns

Index(['calitp_itp_id', 'stop_id', 'stop_lat', 'stop_lon', 'stop_name',
       'route_type', 'geometry', 'hqta_type'],
      dtype='object')

In [6]:
# geoparquet_gcs_export(bus_hqtc, 'bus_hqtc')

In [7]:
# geoparquet_gcs_export(rail_ferry_brt_stops, 'rail_ferry_brt_stops')

### High Quality Transit Areas Relevant Statutes

[PRC 21155](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?sectionNum=21155.&lawCode=PRC)
* _(3) be within one-half mile of a major transit stop or high-quality transit corridor included in a regional transportation plan._
* Major transit stop definition: _A major transit stop is as defined in Section 21064.3, except that, for purposes of this section, it also includes major transit stops that are included in the applicable regional transportation plan_
* High-quality transit corridor definition: _For purposes of this section, a high-quality transit corridor means a corridor with fixed route bus service with service intervals no longer than 15 minutes during peak commute hours._
    * Unable to locate definition of "peak commute hours"

[PRC 21064.3](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?sectionNum=21064.3.&lawCode=PRC)
* _Major transit stop means a site containing any of the following:
(a) An existing rail or bus rapid transit station.
(b) A ferry terminal served by either a bus or rail transit service.
(c) The intersection of two or more major bus routes with a frequency of service interval of 15 minutes or less during the morning and afternoon peak commute periods._
    * "Intersection" may not be sufficiently well-defined for this analysis

[PRC 21060.2](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?lawCode=PRC&sectionNum=21060.2.&highlight=true&keyword=bus%20rapid%20transit)
* _(a) “Bus rapid transit” means a public mass transit service provided by a public agency or by a public-private partnership that includes all of the following features:
(1) Full-time dedicated bus lanes or operation in a separate right-of-way dedicated for public transportation with a frequency of service interval of 15 minutes or less during the morning and afternoon peak commute periods.
(2) Transit signal priority.
(3) All-door boarding.
(4) Fare collection system that promotes efficiency.
(5) Defined stations._
    * Unlikely to determine if a service qualifies as BRT under this definition using GTFS alone

## Bus Major Stops

In [8]:
bus_hqtc.head(3)

,hq_transit_corr,shape_id,geometry,calitp_itp_id,hqta_segment_id,segment_sequence,stop_id,am_max_trips,pm_max_trips,departure_hour,n,hqta_type,route_type
2755,True,0500,"MULTIPOLYGON (((212402.267 -486786.382, 212428...",142,2197551287,0,3977,10.0,9.0,NaN,NaN,hq_transit_corr,3
2756,True,0843,"MULTIPOLYGON (((200815.727 -472198.441, 200811...",142,1555422069,0,6679,5.0,6.0,NaN,NaN,hq_transit_corr,3
2758,True,10,"MULTIPOLYGON (((170484.804 -424390.901, 170484...",243,3850303330,1,147,10.0,12.0,NaN,NaN,hq_transit_corr,3


In [9]:
gdf = bus_hqtc
output = gpd.GeoDataFrame()

def find_intersections(row):
    global output
    # display(row)
    # display(row.shape_id)
    this_row = gdf >> filter(_.shape_id == row.shape_id)
    not_this_row = gdf >> filter(_.shape_id != row.shape_id)
    # print(type(filtered))
    clip_row = gpd.clip(this_row, not_this_row)
    output = pd.concat((output, clip_row))
    return

In [10]:
from tqdm import tqdm

In [11]:
tqdm.pandas()

In [20]:
## ran 5/17
# _test = bus_hqtc.progress_apply(find_intersections, axis=1)

In [21]:
# output.geometry = output.geometry.buffer(50)

In [22]:
# shared_utils.utils.geoparquet_gcs_export(output, f'{GCS_FILE_PATH}', f'major_bus_stops_working')

In [23]:
major_bus = gpd.read_parquet(f'{GCS_FILE_PATH}major_bus_stops_working.parquet')

In [24]:
# map_hqta(major_bus)

In [25]:
def drop_big_areas(geometry):
    if type(geometry) == shapely.geometry.multipolygon.MultiPolygon:
        filtered = [x for x in list(geometry.geoms) if x.length < 1000]
        if len(filtered) > 0:
            return shapely.geometry.MultiPolygon(filtered)
    elif type(geometry) == shapely.geometry.polygon.Polygon:
        if geometry.length < 1000:
            return geometry
    else:
        return np.nan

In [26]:
major_bus['geometry'] = major_bus.geometry.apply(drop_big_areas)

In [27]:
row_per_stop = gpd.GeoDataFrame()
def explode_geoms(row):
    global row_per_stop
    if type(row.geometry) == shapely.geometry.multipolygon.MultiPolygon:
        for geom in row.geometry.geoms:
            row.geometry = geom.centroid
            row_per_stop = pd.concat((row_per_stop, row))
    elif type(row.geometry) == shapely.geometry.polygon.Polygon:
        row.geometry = row.geometry.centroid
        row_per_stop = pd.concat((row_per_stop, row))
    
    return row

In [28]:
major_bus = major_bus.dropna(subset=['geometry'])

major_bus.apply(explode_geoms, axis=1)

row_per_stop = row_per_stop.reset_index(drop=True)

row_per_stop = row_per_stop[['calitp_itp_id', 'stop_id', 'geometry']]

row_per_stop['hqta_type'] = 'major_transit_stop'

row_per_stop = row_per_stop.set_crs(
    shared_utils.geography_utils.CA_NAD83Albers)

/tmp/ipykernel_1131/465189961.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
/tmp/ipykernel_1131/465189961.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
/tmp/ipykernel_1131/465189961.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
/tmp/ipykernel_1131/465189961.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
/tmp/ipykernel_1131/465189961.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
/tmp/ipykernel_1131/465189961.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

AttributeError: 'DataFrame' object has no attribute 'set_crs'

In [33]:
row_per_stop = gpd.GeoDataFrame(row_per_stop, geometry=row_per_stop.geometry,
                               crs = shared_utils.geography_utils.CA_NAD83Albers)

In [35]:
shared_utils.utils.geoparquet_gcs_export(row_per_stop, f'{GCS_FILE_PATH}', f'major_bus_stops')

/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:38: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')


In [36]:
major_bus_stops = gpd.read_parquet(f'{GCS_FILE_PATH}major_bus_stops.parquet')

In [37]:
major_bus_stops.head(3)

,calitp_itp_id,stop_id,geometry,hqta_type
0,142,3977,POINT (212896.375 -487200.839),major_transit_stop
1,142,3977,POINT (212351.822 -486889.713),major_transit_stop
2,142,3977,POINT (198974.208 -471136.313),major_transit_stop


In [38]:
analysis_date = dt.date(2022, 5, 4) ## Wed, May 4

In [40]:
tbl_stops = (tbl.views.gtfs_schedule_fact_daily_feed_stops()
 >> filter(_.date == analysis_date)
 >> filter(_.calitp_extracted_at < analysis_date)
 >> filter(_.calitp_deleted_at > analysis_date)
 >> select(_.stop_key)
 >> inner_join(_, tbl.views.gtfs_schedule_dim_stops(), on = 'stop_key')
 >> select(_.stop_id, _.stop_lat, _.stop_lon, _.calitp_itp_id)
 >> filter(_.calitp_itp_id != 200)
 >> collect()
)

In [41]:
tbl_stops = gpd.GeoDataFrame(tbl_stops,
                 geometry = gpd.points_from_xy(tbl_stops.stop_lon, tbl_stops.stop_lat),
                 crs = 'EPSG:4326').to_crs(shared_utils.geography_utils.CA_NAD83Albers)

In [42]:
tbl_stops.head(3)

,stop_id,stop_lat,stop_lon,calitp_itp_id,geometry
0,31795,37.791443,-122.477299,257,POINT (-217835.170 -22157.798)
1,268762,37.792844,-122.475306,257,POINT (-217655.895 -22006.683)
2,382975,37.799203,-122.477796,257,POINT (-217856.320 -21294.296)


In [43]:
to_join = major_bus_stops[['calitp_itp_id']].astype({'calitp_itp_id': 'int64'})

In [44]:
## all stops for an operator with at least 1 major stop
tbl_stops_major = (tbl_stops
         >> inner_join(_, to_join, on=['calitp_itp_id'])
         >> distinct(_.calitp_itp_id, _.stop_id, _keep_all=True)
            )


In [45]:
tbl_stops_major.head(3)

,stop_id,stop_lat,stop_lon,calitp_itp_id,geometry
0,3846,37.948393,-122.364974,4,POINT (-207525.065 -4967.813)
1,4092,37.919806,-122.326412,4,POINT (-204219.929 -8228.504)
2,3054,37.816597,-122.205524,4,POINT (-193876.840 -19952.457)


In [46]:
tbl_stops_major.geometry = tbl_stops_major.buffer(100)

In [47]:
tbl_stops_major.head(3)

,stop_id,stop_lat,stop_lon,calitp_itp_id,geometry
0,3846,37.948393,-122.364974,4,"POLYGON ((-207425.065 -4967.813, -207425.546 -..."
1,4092,37.919806,-122.326412,4,"POLYGON ((-204119.929 -8228.504, -204120.410 -..."
2,3054,37.816597,-122.205524,4,"POLYGON ((-193776.840 -19952.457, -193777.321 ..."


In [48]:
drop_id = major_bus_stops.drop(columns=['stop_id'])

In [49]:
spatial_stops_major = tbl_stops_major.sjoin(drop_id, how='inner', predicate='contains')

In [50]:
new_major_stops = spatial_stops_major.drop_duplicates(subset=['stop_id', 'calitp_itp_id_left', 'calitp_itp_id_right'])

In [51]:
new_major_stops.geometry = new_major_stops.centroid

/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [53]:
# map_hqta(new_major_stops)

### Bus Corridors to Stops Along Corridor

In [54]:
tbl_stops_corridors = tbl_stops

In [55]:
tbl_stops_corridors.head(3)

,stop_id,stop_lat,stop_lon,calitp_itp_id,geometry
0,31795,37.791443,-122.477299,257,POINT (-217835.170 -22157.798)
1,268762,37.792844,-122.475306,257,POINT (-217655.895 -22006.683)
2,382975,37.799203,-122.477796,257,POINT (-217856.320 -21294.296)


In [58]:
bus_hqtc.head(3)

,hq_transit_corr,shape_id,geometry,calitp_itp_id,hqta_segment_id,segment_sequence,stop_id,am_max_trips,pm_max_trips,departure_hour,n,hqta_type,route_type
2755,True,0500,"MULTIPOLYGON (((212402.267 -486786.382, 212428...",142,2197551287,0,3977,10.0,9.0,NaN,NaN,hq_transit_corr,3
2756,True,0843,"MULTIPOLYGON (((200815.727 -472198.441, 200811...",142,1555422069,0,6679,5.0,6.0,NaN,NaN,hq_transit_corr,3
2758,True,10,"MULTIPOLYGON (((170484.804 -424390.901, 170484...",243,3850303330,1,147,10.0,12.0,NaN,NaN,hq_transit_corr,3


In [59]:
stops_in_corridor = tbl_stops_corridors.clip(bus_hqtc)

In [61]:
stops_in_corridor.drop_duplicates(subset = ['stop_id', 'calitp_itp_id']).info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 33188 entries, 108932 to 55220
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   stop_id        33188 non-null  object  
 1   stop_lat       33188 non-null  float64 
 2   stop_lon       33188 non-null  float64 
 3   calitp_itp_id  33188 non-null  int64   
 4   geometry       33188 non-null  geometry
dtypes: float64(2), geometry(1), int64(1), object(1)
memory usage: 1.5+ MB


In [69]:
# map_hqta(bus_hqtc)

In [70]:
# stop_subset = stops_in_corridor >> head(2000)

In [71]:
# map_hqta(stop_subset)

#### Definitions and Output:

* hqta_type: major_transit_stop
* stop_id: one stop id... (not ideal, but oh well)
* geometry: .buffer(700)?
* _can alway pull more info spatially_

## Unbuffered Export

Thank you for this data. It would be useful for us to get the HQTC stops as a point data file, not a polygon. Also, if you could differentiate between train, bus, BRT, and ferry stop that would be immensely helpful.

Let me know if it is possible to get the data in this format.  

#### Major Transit Stops (bus intersections)

In [72]:
new_major_stops.head(3)

,stop_id,stop_lat,stop_lon,calitp_itp_id_left,geometry,index_right,calitp_itp_id_right,hqta_type
18,181,37.765141,-122.242130,4,POINT (-197229.364 -25595.579),2068,4,major_transit_stop
469,59255,37.765368,-122.242085,4,POINT (-197224.809 -25570.445),2068,4,major_transit_stop
628,147,37.765284,-122.241692,4,POINT (-197190.466 -25580.597),2068,4,major_transit_stop


In [74]:
new_major_bus_extract = new_major_stops >> select(_.calitp_itp_id_primary == _.calitp_itp_id_left, _.stop_id,
                          _.calitp_itp_id_secondary == _.calitp_itp_id_right, _.hqta_type, _.geometry
                         )
new_major_bus_extract['hqta_type'] = 'major_stop_bus'
new_major_bus_extract.head(3)

,calitp_itp_id_primary,stop_id,calitp_itp_id_secondary,hqta_type,geometry
18,4,181,4,major_stop_bus,POINT (-197229.364 -25595.579)
469,4,59255,4,major_stop_bus,POINT (-197224.809 -25570.445)
628,4,147,4,major_stop_bus,POINT (-197190.466 -25580.597)


#### Stops Along HQ Transit Corridors (bus)

In [75]:
stops_in_corridor.head(3)

,stop_id,stop_lat,stop_lon,calitp_itp_id,geometry
108932,5943533,34.154372,-119.177909,231,POINT (75799.261 -428713.584)
110220,3290234,34.154601,-119.180248,231,POINT (75583.384 -428690.047)
64957,PVRCST:2,34.154601,-119.180248,123,POINT (75583.400 -428690.022)


In [99]:
stops_extract = stops_in_corridor >> select(_.calitp_itp_id_primary == _.calitp_itp_id,
                                            _.stop_id, _.geometry)

In [100]:
stops_extract['hqta_type'] = 'hq_corridor_bus'

In [101]:
stops_extract.head(3)

,calitp_itp_id_primary,stop_id,geometry,hqta_type
108932,231,5943533,POINT (75799.261 -428713.584),hq_corridor_bus
110220,231,3290234,POINT (75583.384 -428690.047),hq_corridor_bus
64957,123,PVRCST:2,POINT (75583.400 -428690.022),hq_corridor_bus


#### Major Transit Stops (rail/ferry/brt)

In [102]:
rail_ferry_brt_stops.head(3)

,calitp_itp_id,stop_id,stop_lat,stop_lon,stop_name,route_type,geometry,hqta_type
101,278,75005,32.569515,-117.066965,Iris Avenue Station,0,POINT (275828.261 -600432.937),major_transit_stop
35,278,75004,32.569762,-117.067176,Iris Avenue Station,0,POINT (275807.549 -600406.211),major_transit_stop
86,278,75007,32.584454,-117.083860,Palm Avenue Station,0,POINT (274189.052 -598828.705),major_transit_stop


In [111]:
rail_ferry_brt_extract = rail_ferry_brt_stops >> select(_.calitp_itp_id == _.calitp_itp_id, _.stop_id,
                                      _.hqta_type, _.route_type, _.geometry
                         )

In [112]:
rail_ferry_brt_extract = (rail_ferry_brt_extract
    >> mutate(
    hqta_type = case_when({
        _.route_type.isin(['0', '1', '2']): 'major_stop_rail',
        _.route_type == '3': 'major_stop_brt',
        _.route_type == '4': 'major_stop_ferry'
        })
    )
    >> select(-_.route_type, _.calitp_itp_id_primary == _.calitp_itp_id)
)

In [113]:
rail_ferry_brt_extract.head(3)

,calitp_itp_id_primary,stop_id,hqta_type,geometry
101,278,75005,major_stop_rail,POINT (275828.261 -600432.937)
35,278,75004,major_stop_rail,POINT (275807.549 -600406.211)
86,278,75007,major_stop_rail,POINT (274189.052 -598828.705)


In [114]:
points_combined = pd.concat((new_major_bus_extract, stops_extract, rail_ferry_brt_extract))

In [115]:
points_combined.dtypes

calitp_itp_id_primary         int64
stop_id                      object
calitp_itp_id_secondary      object
hqta_type                    object
geometry                   geometry
dtype: object

In [121]:
points_combined

,calitp_itp_id_primary,stop_id,calitp_itp_id_secondary,hqta_type,geometry
18,4,181,4,major_stop_bus,POINT (-197229.364 -25595.579)
469,4,59255,4,major_stop_bus,POINT (-197224.809 -25570.445)
628,4,147,4,major_stop_bus,POINT (-197190.466 -25580.597)
2927,4,55546,4,major_stop_bus,POINT (-197179.343 -25545.611)
5092,4,182,4,major_stop_bus,POINT (-197179.343 -25545.611)
...,...,...,...,...,...
13,127,43008,NaN,major_stop_ferry,POINT (-210476.942 -21780.838)
14,127,43003,NaN,major_stop_ferry,POINT (-217668.884 -14964.633)
16,127,43007,NaN,major_stop_ferry,POINT (-215721.361 -13173.100)
9,280,890004,NaN,major_stop_ferry,POINT (-207130.442 -9301.491)


In [161]:
# map_hqta(points_combined)

In [126]:
names = (tbl.views.gtfs_schedule_dim_feeds()
    >> filter(_.calitp_extracted_at < analysis_date, _.calitp_deleted_at > analysis_date)
    >> select(_.calitp_itp_id_primary == _.calitp_itp_id, _.agency_name_primary == _.calitp_agency_name)
    >> collect()
)
with_names = points_combined >> inner_join(_, names, on = 'calitp_itp_id_primary')

In [127]:
with_names = with_names.drop_duplicates(subset=['calitp_itp_id_primary', 'hqta_type', 'stop_id'])

In [128]:
with_names.head(3)

,calitp_itp_id_primary,stop_id,calitp_itp_id_secondary,hqta_type,geometry,agency_name_primary
0,4,181,4,major_stop_bus,POINT (-197229.364 -25595.579),AC Transit
2,4,59255,4,major_stop_bus,POINT (-197224.809 -25570.445),AC Transit
4,4,147,4,major_stop_bus,POINT (-197190.466 -25580.597),AC Transit


In [129]:
with_names >> count(_.hqta_type)

,hqta_type,n
0,hq_corridor_bus,33188
1,major_stop_brt,207
2,major_stop_bus,5431
3,major_stop_ferry,18
4,major_stop_rail,1439


In [130]:
analysis_date

datetime.date(2022, 5, 4)

In [131]:
shared_utils.utils.geoparquet_gcs_export(with_names, f'{GCS_FILE_PATH}/export/2022-05-04', f'hq_transit_stops')

/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:38: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')


In [132]:
with_names.to_file('./ca_high_quality_transit_point.geojson', driver='GeoJSON')

/opt/conda/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


### Combining and Buffering

* General buffer distance: 1/2mi ~= 805 meters
* Bus corridors are already buffered 50 meters, so will buffer 755 meters

In [162]:
major_stops_named = with_names >> filter(_.hqta_type != 'hq_corridor_bus')

In [163]:
bus_hqtc.geometry = bus_hqtc.geometry.buffer(755)
major_stops_named.geometry = major_stops_named.buffer(805)
# row_per_stop.geometry = row_per_stop.geometry.buffer(805)
# rail_ferry_brt_stops.geometry = rail_ferry_brt_stops.geometry.buffer(805)

/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [177]:
major_stops_named >> head(2)

,calitp_itp_id_primary,stop_id,calitp_itp_id_secondary,hqta_type,geometry,agency_name_primary
0,4,181,4,major_stop_bus,"POLYGON ((-196424.364 -25595.579, -196428.240 ...",AC Transit
2,4,59255,4,major_stop_bus,"POLYGON ((-196419.809 -25570.445, -196423.686 ...",AC Transit


In [178]:
bus_hqtc >> head(2)

,hq_transit_corr,shape_id,geometry,calitp_itp_id,hqta_segment_id,segment_sequence,stop_id,am_max_trips,pm_max_trips,departure_hour,n,hqta_type,route_type
2755,True,0500,"MULTIPOLYGON (((211656.381 -486108.779, 211648...",142,2197551287,0,3977,10.0,9.0,NaN,NaN,hq_transit_corr,3
2756,True,0843,"MULTIPOLYGON (((199947.647 -472521.650, 199947...",142,1555422069,0,6679,5.0,6.0,NaN,NaN,hq_transit_corr,3


In [214]:
bus_hqtc_formatted = bus_hqtc >> select(_.calitp_itp_id_primary == _.calitp_itp_id, _.stop_id, _.hqta_segment_id,
                                        _.am_max_trips, _.pm_max_trips, _.hqta_type, _.geometry)
bus_hqtc_formatted.hqta_type = 'hq_corridor_bus'
bus_hqtc_formatted.calitp_itp_id_primary = bus_hqtc_formatted.calitp_itp_id_primary.astype('int64')
bus_hqtc_formatted = bus_hqtc_formatted >> inner_join(_, names, on = 'calitp_itp_id_primary')

In [216]:
all_hqta = pd.concat([major_stops_named, bus_hqtc_formatted])

In [217]:
all_hqta.columns

Index(['calitp_itp_id_primary', 'stop_id', 'calitp_itp_id_secondary',
       'hqta_type', 'geometry', 'agency_name_primary', 'hqta_segment_id',
       'am_max_trips', 'pm_max_trips'],
      dtype='object')

In [218]:
all_hqta

,calitp_itp_id_primary,stop_id,calitp_itp_id_secondary,hqta_type,geometry,agency_name_primary,hqta_segment_id,am_max_trips,pm_max_trips
0,4,181,4,major_stop_bus,"POLYGON ((-196424.364 -25595.579, -196428.240 ...",AC Transit,NaN,NaN,NaN
2,4,59255,4,major_stop_bus,"POLYGON ((-196419.809 -25570.445, -196423.686 ...",AC Transit,NaN,NaN,NaN
4,4,147,4,major_stop_bus,"POLYGON ((-196385.466 -25580.597, -196389.343 ...",AC Transit,NaN,NaN,NaN
6,4,55546,4,major_stop_bus,"POLYGON ((-196374.343 -25545.611, -196378.220 ...",AC Transit,NaN,NaN,NaN
8,4,182,4,major_stop_bus,"POLYGON ((-196374.343 -25545.611, -196378.220 ...",AC Transit,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
792,4,2874,NaN,hq_corridor_bus,"MULTIPOLYGON (((-199451.077 -19885.823, -19940...",AC Transit,3.672190e+09,7.0,6.0
793,293,139,NaN,hq_corridor_bus,"MULTIPOLYGON (((18757.773 -397024.907, 18777.7...",Santa Barbara Metropolitan Transit District,1.180391e+09,8.0,9.0
794,293,608,NaN,hq_corridor_bus,"POLYGON ((26470.781 -399176.692, 26461.762 -39...",Santa Barbara Metropolitan Transit District,2.198401e+09,8.0,8.0
795,293,793,NaN,hq_corridor_bus,"MULTIPOLYGON (((150130.017 -451891.121, 150130...",Santa Barbara Metropolitan Transit District,3.386448e+08,7.0,8.0


In [221]:
tbl.gtfs_schedule.stops() >> filter(_.calitp_itp_id == 300, _.stop_id == '689')

,calitp_itp_id,calitp_url_number,stop_id,tts_stop_name,stop_lat,stop_lon,zone_id,parent_station,stop_code,stop_name,stop_desc,stop_url,location_type,stop_timezone,wheelchair_boarding,level_id,platform_code,calitp_extracted_at,calitp_hash,stop_key
0,300,0,689,None,34.043796,-118.431692,None,None,2279,WESTWOOD SB & OLYMPIC FS,2200 WESTWOOD BLVD & W OLYMPIC BLVD,None,None,None,0,None,None,2022-03-08,sMHUc7bKuKlJjfNOiYQ88Q==,-6361307378120019887


In [222]:
tbl.gtfs_schedule.stops() >> filter(_.stop_id == '315608')

,calitp_itp_id,calitp_url_number,stop_id,tts_stop_name,stop_lat,stop_lon,zone_id,parent_station,stop_code,stop_name,stop_desc,stop_url,location_type,stop_timezone,wheelchair_boarding,level_id,platform_code,calitp_extracted_at,calitp_hash,stop_key
0,290,0,315608,None,37.43476,-122.417448,1,None,315608,Miramontes Point Rd-Moonridge Apts,None,None,0,None,None,None,None,2021-04-16,KGF5/ji8KsbHSE6Q8tBhoA==,5426997252713801279
1,200,0,315608,None,37.43476,-122.417448,SM:66204,None,315608,Miramontes Point Rd-Moonridge Apts,None,None,0,America/Los_Angeles,0,None,None,2022-05-05,VSF7Vu7ucMiJkofTTRnlLg==,732130110778268565
2,290,1,315608,None,37.43476,-122.417448,66204,None,315608,Miramontes Point Rd-Moonridge Apts,None,None,0,None,None,None,None,2022-05-04,6hqqRJG99QBlUknDwqbE2w==,-8455697345374201487


In [219]:
map_hqta(bus_hqtc_formatted, 'stop_id')

Map(center=[33.665997215098606, -117.76439560705839], controls=(ZoomControl(options=['position', 'zoom_in_text…

In [30]:
all_hqta = all_hqta[['calitp_itp_id', 'geometry', 'hqta_type']]
all_hqta = all_hqta.reset_index(drop=True)
all_hqta['calitp_itp_id'] = all_hqta['calitp_itp_id'].astype('int64')

### Format for export

In [31]:
agency_names = (tbl.gtfs_schedule.agency()
 >> select(_.calitp_itp_id, _.agency_name)
 >> collect()
)

In [32]:
all_hqta = all_hqta >> inner_join(_, agency_names, on='calitp_itp_id')

In [39]:
all_hqta = gpd.read_parquet(f'{GCS_FILE_PATH}ca_high_quality_transit.parquet')

In [16]:
metro = bus_hqtc >> filter(_.calitp_itp_id == '182')

In [18]:
map_hqta(metro, 'pm_max_trips')

Map(center=[34.078792878404975, -118.30956119336547], controls=(ZoomControl(options=['position', 'zoom_in_text…

In [ ]:
# geoparquet_gcs_export(all_hqta, 'ca_high_quality_transit')
# all_hqta.to_file('./ca_high_quality_transit.geojson', driver='GeoJSON')
# all_hqta.to_file('./ca_high_quality_transit/ca_high_quality_transit.shp')

## Static Map Images (all HQTAs)

![bay](img/bay_valley_all.png)

![fresno](img/fres_all.png)

![san_diego](img/sd_all.png)

In [ ]:
# map_hqta(all_hqta)

In [15]:
all_hqta

,calitp_itp_id,geometry,hqta_type,agency_name
0,48,"POLYGON ((-133534.279 167527.858, -133517.826 ...",hq_transit_corr,B-Line
1,48,"MULTIPOLYGON (((-157718.392 195993.182, -15773...",hq_transit_corr,B-Line
2,142,"POLYGON ((211656.399 -486108.638, 211648.381 -...",hq_transit_corr,Orange County Transportation Authority
3,142,"MULTIPOLYGON (((199947.647 -472521.650, 199947...",hq_transit_corr,Orange County Transportation Authority
4,142,"MULTIPOLYGON (((187094.291 -451836.591, 187091...",hq_transit_corr,Orange County Transportation Authority
...,...,...,...,...
31942,361,"POLYGON ((62774.123 -187182.324, 62770.247 -18...",major_transit_stop,Visalia Transit
31943,301,"POLYGON ((-235898.301 50349.674, -235902.177 5...",major_transit_stop,Santa Rosa CityBus
31944,301,"POLYGON ((-235898.301 50349.674, -235902.177 5...",major_transit_stop,Santa Rosa CityBus
31945,201,"POLYGON ((-209333.821 -24661.220, -209337.698 ...",major_transit_stop,Mission Bay TMA
